# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

from collections import deque
from d4pg_agent import Agent
from params import Params

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [3]:
env = UnityEnvironment(file_name='Reacher_Windows_multi_agent_x86_64/Reacher.exe')
#env = UnityEnvironment(file_name='Reacher_Windows_single_agent_x86_64/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [6]:
# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [7]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [8]:
def d4pg(scores, n_episodes=1000, max_t=300, print_every=100, save_every=100, target_score=1000.0, N_step_bootstrap=5, prefill_memory_qty=0):
    
    env_info = env.reset(train_mode=True)[brain_name]
    scores_deque = deque(maxlen=print_every)
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations              # get the current state
        agent.reset()
        score = 0
                
        for t in range(max_t):                             
            
            # Perform actions from each agent's policy network (clipped actions [0, -1])
            actions = [ agent.act(state) for state in states ]  
            actions = np.stack(actions)            
            env_info = env.step(actions)[brain_name]      
            next_states, rewards, dones = env_info.vector_observations, env_info.rewards, env_info.local_done
            
            agent.step(states, actions, rewards, next_states, dones, prefill_memory_qty, clear_nstep_buffer=(t==0))
               
            states = next_states
            score += (sum(rewards) / len(rewards))  # ALT METRIC: Max
            
            # Prevent agent from learning NaN rewards
            has_NAN_in_rewards = np.isnan(rewards)
            if any(has_NAN_in_rewards):
                rewards[has_NAN_in_rewards] = -5
                print("NaN Reward Found! Converting to R=-5 instead...")
            
            # Break if any of the agent is done
            if any(dones):
                break 
            
        # Print crucial results for progress tracking
        if agent.memory_buffer_prefilled(prefill_memory_qty):
            
            scores_deque.append(score)
            scores.append(score)
            print('\rEpisode {}\ Score: {:.2f}'.format(i_episode, scores_deque[-1]), end="")

            if i_episode % save_every == 0:
                torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
                torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')

            if i_episode % print_every == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

                if np.mean(scores_deque) >= target_score:
                    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'
                          .format(i_episode, np.mean(scores_deque)))
                    break

    return scores

In [9]:
def plot_graph(scores):
    # Plot a graph
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
##################
scores = []
args = Params()
args.lr_actor = 5e-4
args.lr_critic = 1e-3
args.hard_weights_update_every = 350
agent = Agent(state_size, action_size, random_seed=0, params=args)
d4pg(scores, n_episodes=200, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=5, prefill_memory_qty=5000)
plot_graph(scores)


=============== HYPERPARAMS ===============
DEVICE:  cuda:0
BUFFER_SIZE:  300000
BATCH_SIZE:  128
GAMMA:  0.99
TAU:  0.001
LR_ACTOR:  0.0005
LR_CRITIC:  0.001
WEIGHT_DECAY:  0.0001
HARD_UPDATE:  True
LEARN_EVERY:  1
HARD_WEIGHTS_UPDATE_EVERY:  350
N_STEP_BOOTSTRAP:  5
VMIN:  0
VMAX:  0.3
NUM_ATOMS:  100

Prefilling Replay Memory Buffer: 5000 / 5000
 ===== Replay Buffer prefilled. Agent will begin learning. =====

Episode 2	Average Score: 0.15
Episode 4	Average Score: 0.36
Episode 6	Average Score: 1.26
Episode 8	Average Score: 2.25
Episode 10	Average Score: 3.11
Episode 12	Average Score: 4.81


In [ ]:
############################## PARAM SWEEP ##############################
print("CUDA AVAILABLE: ", torch.cuda.is_available())

# ##################
# scores = []
# args = Params()
# args.lr_actor = 1e-4
# args.lr_critic = 1e-4
# args.hard_weights_update_every = 350
# agent = Agent(state_size, action_size, random_seed=0, params=args)
# d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=5, prefill_memory_qty=5000)
# plot_graph(scores)

# ##################
# scores = []
# args = Params()
# args.lr_actor = 1e-4
# args.lr_critic = 4e-4
# args.hard_weights_update_every = 350
# agent = Agent(state_size, action_size, random_seed=0, params=args)
# d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=5, prefill_memory_qty=5000)
# plot_graph(scores)

##################
scores = []
args = Params()
args.lr_actor = 5e-4
args.lr_critic = 5e-4
args.hard_weights_update_every = 350
agent = Agent(state_size, action_size, random_seed=0, params=args)
d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=5, prefill_memory_qty=5000)
plot_graph(scores)

##################
scores = []
args = Params()
args.lr_actor = 5e-4
args.lr_critic = 7.5e-4
args.hard_weights_update_every = 350
agent = Agent(state_size, action_size, random_seed=0, params=args)
d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=5, prefill_memory_qty=5000)
plot_graph(scores)

In [ ]:
# ########################################################################################
# # VARYING HARD UPDATE RATES
# ########################################################################################

##################
scores = []
args = Params()
args.lr_actor = 1e-4
args.lr_critic = 1e-4
args.hard_weights_update_every = 350
agent = Agent(state_size, action_size, random_seed=0, params=args)
d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=1, prefill_memory_qty=5000)
plot_graph(scores)

##################
scores = []
args = Params()
args.lr_actor = 1e-4
args.lr_critic = 4e-4
args.hard_weights_update_every = 350
agent = Agent(state_size, action_size, random_seed=0, params=args)
d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=1, prefill_memory_qty=5000)
plot_graph(scores)

##################
scores = []
args = Params()
args.lr_actor = 5e-4
args.lr_critic = 5e-4
args.hard_weights_update_every = 350
agent = Agent(state_size, action_size, random_seed=0, params=args)
d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=1, prefill_memory_qty=5000)
plot_graph(scores)

##################
scores = []
args = Params()
args.lr_actor = 5e-4
args.lr_critic = 7.5e-4
args.hard_weights_update_every = 350
agent = Agent(state_size, action_size, random_seed=0, params=args)
d4pg(scores, n_episodes=50, max_t=1000, print_every=2, save_every=100, target_score=30.0, N_step_bootstrap=1, prefill_memory_qty=5000)
plot_graph(scores)

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
# # load the weights from file
# agent = Agent(state_size, action_size, random_seed=1)
# agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
# agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))


# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment
# states = env_info.vector_observations                # get the current state
# score = 0                                              # initialize the score
# while True:
#     # Perform actions from each agent's policy network (clipped actions [0, -1])
#     actions = [ agent.act(state, add_noise=False) for state in states ]  
#     actions = np.stack(actions)      
#     env_info = env.step(actions)[brain_name]  # send the action to the environment (BUG: Cast action to int)
#     next_states = env_info.vector_observations       # get the next state
#     rewards = env_info.rewards                       # get the reward
#     dones = env_info.local_done                      # see if episode has finished
#     score += max(rewards)                                    # update the score
#     states = next_states                                 # roll over the state to next time step
#     if max(dones):                                           # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
# env.close()